In [1]:
!pip install --upgrade --quiet transformers accelerate bitsandbytes peft safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.6 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [ ]:
from huggingface_hub import HfApi
api = HfApi('HF_TOKEN')
api

In [ ]:
HF_TOKEN = "HF_TOKEN"  # replace with your token

MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=HF_TOKEN)
# Ensure pad token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
# --- 5️⃣ 4-bit Quantization Config ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"
)

# --- 6️⃣ Load Model in 4-bit ---
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    use_auth_token=HF_TOKEN
)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
!pip install peft

In [8]:
# PEFT imports
from peft import LoraConfig, get_peft_model, TaskType


# --- 3️⃣ Setup LoRA ---
lora_config = LoraConfig(
    r=16,                       # low-rank dimension
    lora_alpha=32,               # scaling
    target_modules=["q_proj","v_proj"],  # attention layers
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [9]:
from datasets import load_dataset
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
from google.colab import drive
drive.mount('/content/drive')  # remount if needed

import sys
sys.path.append("/content/drive/MyDrive/chatbot/final_finance_dataset.jsonl")

print('Success')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Success


In [11]:
!ls /content/drive/MyDrive/chatbot

final_finance_dataset.jsonl  llama2-lora-finance


In [ ]:
from datasets import load_dataset

# Load JSONL file
dataset = load_dataset("json", data_files="LOCATION_HIDDEN")

# Inspect the dataset
print(dataset)
print(dataset['train'][0])


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2690
    })
})
{'text': '[/INST] Can someone explain how government bonds work? [/INST] The short of it is that bonds are valued based on a fundamental concept of finance called the "time value of money" </s>'}


In [13]:
# LLaMA-2 tokenizer doesn't always have pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [14]:

# Tokenize function
def tokenize_function(example):
    tokenized = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # this is key
    return tokenized

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

print("Tokenization complete. Sample:", tokenized_dataset['train'][0])

Map:   0%|          | 0/2690 [00:00<?, ? examples/s]

Tokenization complete. Sample: {'text': '[/INST] Can someone explain how government bonds work? [/INST] The short of it is that bonds are valued based on a fundamental concept of finance called the "time value of money" </s>', 'input_ids': [1, 518, 29914, 25580, 29962, 1815, 4856, 5649, 920, 5874, 289, 13788, 664, 29973, 518, 29914, 25580, 29962, 450, 3273, 310, 372, 338, 393, 289, 13788, 526, 659, 6742, 2729, 373, 263, 15281, 6964, 310, 1436, 749, 2000, 278, 376, 2230, 995, 310, 6909, 29908, 29871, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [ ]:


from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/LOCATION/llama2-lora-finance-final",
    per_device_train_batch_size=1,        # keep 1 for T4 memory limits
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    fp16=True,
    logging_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=1,
    optim="adamw_torch",
    dataloader_num_workers=2
)



In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer
)




/tmp/ipython-input-3364810116.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
model.save_pretrained("/LOCATION/llama2-lora-finance-final")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aayushpandey7310 (aayushpandey7310-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,1.124800
200,0.249800
300,0.230100
400,0.230500
500,0.207000
600,0.244600


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/other.py:1228: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-68fc6191-2b85f57d0ec999172f5b3fcb;45610e8e-db47-4f8d-8aee-d0e340ffb8e7)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Pl

In [ ]:
print("=" * 60)
print("🎉 Training Complete! Now saving and uploading...")
print("=" * 60)


# --- 4️⃣ Upload to Hugging Face ---
from huggingface_hub import HfApi, create_repo, login

# --- 1️⃣ Mount Google Drive (if not already mounted) ---
from google.colab import drive
try:
    drive.mount('/content/drive')
except:
    print("Drive already mounted")

# --- 2️⃣ Save Model to Google Drive ---
save_path = "LOCATION/finance-llama-finetuned-another2"

print(f"\n💾 Saving model to: {save_path}")
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Check folder size
import os
def get_size_mb(path):
    total = sum(
        os.path.getsize(os.path.join(dirpath, filename))
        for dirpath, _, filenames in os.walk(path)
        for filename in filenames
    )
    return total / (1024 * 1024)

size = get_size_mb(save_path)
print(f"✅ Saved to Google Drive! Size: {size:.2f} MB")

# --- 3️⃣ Upload to Hugging Face ---
from huggingface_hub import HfApi, create_repo, login

# --- Log in to Hugging Face ---
HF_USERNAME = "aaysush16"  # your Hugging Face username
HF_TOKEN = "HF_TOKEN"  # your HF token

login(token=HF_TOKEN)

MODEL_NAME = "finance-llama-2-7b-lora"
repo_id = f"{HF_USERNAME}/{MODEL_NAME}"

# Create repo (if it doesn’t exist)
try:
    create_repo(
        repo_id=repo_id,
        repo_type="model",
        exist_ok=True,
        token=HF_TOKEN
    )
    print(f"✅ Repository created/verified")
except Exception as e:
    print(f"Note: {e}")

# Upload the model folder
api = HfApi()
api.upload_folder(
    folder_path=save_path,
    repo_id=repo_id,
    repo_type="model",
    token=HF_TOKEN,
    commit_message="Upload fine-tuned finance model"
)

print("\n" + "=" * 60)
print("🎉 SUCCESS! MODEL UPLOADED!")
print("=" * 60)
print(f"🔗 View at: https://huggingface.co/{repo_id}")
print(f"💾 Size: {size:.2f} MB")
print("\n📝 Next: Create Gradio Space using this model!")
print("=" * 60)


🎉 Training Complete! Now saving and uploading...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

💾 Saving model to: /content/drive/MyDrive/finance-llama-finetuned-another2
✅ Saved to Google Drive! Size: 35.95 MB
✅ Repository created/verified


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...-another2/tokenizer.model: 100%|##########|  500kB /  500kB            

  ...adapter_model.safetensors:  75%|#######4  | 25.2MB / 33.6MB            

No files have been modified since last commit. Skipping to prevent empty commit.



🎉 SUCCESS! MODEL UPLOADED!
🔗 View at: https://huggingface.co/aaysush16/finance-llama-2-7b-lora
💾 Size: 35.95 MB

📝 Next: Create Gradio Space using this model!


In [19]:

# --- 2️⃣ Save Model to Google Drive ---
save_path = "/content/drive/MyDrive/finance-llama-finetuned-anotherr"

print(f"\n💾 Saving model to: {save_path}")
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Check folder size
import os
def get_size_mb(path):
    total = sum(
        os.path.getsize(os.path.join(dirpath, filename))
        for dirpath, _, filenames in os.walk(path)
        for filename in filenames
    )
    return total / (1024 * 1024)

size = get_size_mb(save_path)
print(f"✅ Saved to Google Drive! Size: {size:.2f} MB")


💾 Saving model to: /content/drive/MyDrive/finance-llama-finetuned-anotherr
✅ Saved to Google Drive! Size: 35.95 MB
